# Batching Performance Test for `parse_root_file`

This notebook tests the batching performance in `parse_root_file`, specifically focusing on:
- Different batch sizes
- Batched vs non-batched performance
- Time breakdown for each step (reading, concatenating, zipping)
- Memory usage patterns


In [11]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import awkward as ak
import uproot
from tqdm import tqdm
import gc
import psutil
import tracemalloc
from contextlib import contextmanager
import itertools

from src.parse_atlas import parser
from src.parse_atlas import schemas

%matplotlib inline
plt.style.use('default')


## Find Test Files


In [12]:
# Configuration
input_dir = "/storage/agrp/netalev/data/root_files/"  # Update this path if needed

# Get ROOT files sorted by size
if os.path.exists(input_dir):
    root_files = [f for f in os.listdir(input_dir) if f.endswith(".root")]
    root_files = sorted(root_files, key=lambda x: os.path.getsize(os.path.join(input_dir, x)), reverse=True)
    
    print(f"Found {len(root_files)} ROOT files")
    
    # Show file sizes
    file_sizes = []
    for f in root_files[:5]:  # Top 5 largest
        size_mb = os.path.getsize(os.path.join(input_dir, f)) / (1024**2)
        file_sizes.append({'filename': f, 'size_mb': size_mb})
    
    df_files = pd.DataFrame(file_sizes)
    print("\nTop files to test:")
    print(df_files.to_string(index=False))
    
    # Select test files: one large, one medium, one small
    test_files = []
    if len(root_files) >= 3:
        test_files = [
            (root_files[0], "large"),
            (root_files[len(root_files)//2], "medium"),
            (root_files[-1], "small")
        ]
    elif len(root_files) >= 1:
        test_files = [(root_files[0], "available")]
    
    print(f"\nSelected {len(test_files)} test files")
else:
    print(f"Directory {input_dir} not found. Please update the path.")
    test_files = []


Found 22 ROOT files

Top files to test:
                         filename     size_mb
2024r-pp_mc_b1c18ef224d66cb7.root 2292.645560
   2024r-pp_f2fbf9c5651948e9.root 2147.924682
   2024r-pp_8bc3f9c021427cc5.root 1941.396151
   2024r-pp_9cbe371e887f8663.root 1865.965141
2024r-pp_mc_1e620f3ce6894e8e.root 1580.350282

Selected 3 test files


## Instrumented Version of parse_root_file with Timing


In [13]:
def parse_root_file_instrumented(file_index, tree_names=["CollectionTree"], batch_size=40_000, return_timings=False):
    """
    Instrumented version of parse_root_file that tracks timing for each step.
    """
    timings = {
        'open_file': 0,
        'get_tree': 0,
        'extract_branches': 0,
        'read_batches': 0,
        'concatenate': 0,
        'zip_objects': 0,
        'total': 0
    }
    
    total_start = time.time()
    
    # Open file
    open_start = time.time()
    with uproot.open(file_index) as root_file:
        timings['open_file'] = time.time() - open_start
        
        # Get tree
        tree_start = time.time()
        root_file_keys = root_file.keys()
        tree_name = parser.AtlasOpenParser.get_data_tree_name_for_root_file(root_file_keys, tree_names)
        tree = root_file[tree_name]
        all_tree_branches = set(tree.keys())
        n_entries = tree.num_entries
        is_file_big = n_entries > batch_size if batch_size else False
        timings['get_tree'] = time.time() - tree_start
        
        # Extract branches
        extract_start = time.time()
        obj_branches_and_quantities = parser.AtlasOpenParser.extract_branches_by_obj_in_schema(
            all_tree_branches)
        
        if not obj_branches_and_quantities.values():
            timings['total'] = time.time() - total_start
            return (None, timings) if return_timings else None
        
        all_branches = set(itertools.chain.from_iterable(obj_branches_and_quantities.values()))
        timings['extract_branches'] = time.time() - extract_start
        
        # Initialize storage
        obj_events_by_quantities = {obj_name: [] for obj_name in obj_branches_and_quantities.keys()}
        
        # Define entry ranges
        entry_ranges = []
        if is_file_big:
            entry_ranges = [
                (start, min(start + batch_size, n_entries)) 
                for start in range(0, n_entries, batch_size)
            ]
        else:
            entry_ranges = [(0, n_entries)]
        
        # Read batches
        read_start = time.time()
        for entry_start, entry_stop in entry_ranges:
            batch_data = tree.arrays(all_branches, entry_start=entry_start, entry_stop=entry_stop)
            
            for obj_name, branch_mapping in obj_branches_and_quantities.items():
                subset = batch_data[list(branch_mapping.keys())]
                if len(subset) > 0:
                    obj_events_by_quantities[obj_name].append(subset)
        timings['read_batches'] = time.time() - read_start
        
        # Concatenate
        concat_start = time.time()
        for obj_name, chunks in obj_events_by_quantities.items():
            concatenated = ak.concatenate(chunks)
            obj_events_by_quantities[obj_name] = concatenated
        timings['concatenate'] = time.time() - concat_start
        
        # Zip objects
        zip_start = time.time()
        for obj_name, concatenated in obj_events_by_quantities.items():
            obj_events_by_quantities[obj_name] = ak.zip({
                quantity: concatenated[full_branch]
                for full_branch, quantity in obj_branches_and_quantities[obj_name].items()
            })
        
        result = ak.zip(obj_events_by_quantities, depth_limit=1)
        timings['zip_objects'] = time.time() - zip_start
    
    timings['total'] = time.time() - total_start
    
    return (result, timings) if return_timings else result


In [14]:
def test_batch_sizes(file_path, filename, batch_sizes=[None, 10_000, 20_000, 40_000, 80_000, 160_000]):
    """
    Test parsing with different batch sizes.
    None means no batching (read entire file at once).
    """
    results = []
    
    # First, get file info
    with uproot.open(file_path) as root_file:
        root_file_keys = root_file.keys()
        tree_name = parser.AtlasOpenParser.get_data_tree_name_for_root_file(root_file_keys, ["CollectionTree"])
        tree = root_file[tree_name]
        n_entries = tree.num_entries
        print(tree[tree.keys()[0]] == 0)
        file_size_mb = tree.num_bytes  / (1024**2)
    
    print(f"\n{'='*80}")
    print(f"Testing file: {filename}")
    print(f"File size: {file_size_mb:.2f} MB")
    print(f"Number of entries: {n_entries:,}")
    print(f"{'='*80}")
    
    for batch_size in batch_sizes:
        # Clean up before each test
        gc.collect()
        
        # Get memory before
        process = psutil.Process()
        mem_before = process.memory_info().rss / (1024**2)
        
        # If batch_size is None, use a very large value to effectively disable batching
        effective_batch_size = batch_size if batch_size is not None else n_entries + 1
        
        try:
            start_time = time.time()
            result, timings = parse_root_file_instrumented(
                file_path, 
                batch_size=effective_batch_size,
                return_timings=True
            )
            
            if result is None:
                print(f"⚠️  Batch size {batch_size}: Failed to parse")
                continue
            
            parse_time = time.time() - start_time
            
            # Get memory after
            mem_after = process.memory_info().rss / (1024**2)
            mem_used = mem_after - mem_before
            
            num_events = len(result)
            num_batches = max(1, int(np.ceil(n_entries / effective_batch_size))) if batch_size else 1
            
            result_dict = {
                'filename': filename,
                'file_size_mb': file_size_mb,
                'n_entries': n_entries,
                'batch_size': batch_size if batch_size else 'no_batch',
                'num_batches': num_batches,
                'total_time': timings['total'],
                'open_file_time': timings['open_file'],
                'get_tree_time': timings['get_tree'],
                'extract_branches_time': timings['extract_branches'],
                'read_batches_time': timings['read_batches'],
                'concatenate_time': timings['concatenate'],
                'zip_objects_time': timings['zip_objects'],
                'memory_used_mb': mem_used,
                'num_events': num_events,
                'throughput_mb_per_sec': file_size_mb / timings['total'] if timings['total'] > 0 else 0,
                'time_per_event_ms': (timings['total'] / num_events * 1000) if num_events > 0 else 0
            }
            
            results.append(result_dict)
            
            print(f"\nBatch size: {batch_size if batch_size else 'no_batch':>10}")
            print(f"  Total time: {timings['total']:.2f}s")
            print(f"  ├─ Open file: {timings['open_file']:.3f}s ({timings['open_file']/timings['total']*100:.1f}%)")
            print(f"  ├─ Get tree: {timings['get_tree']:.3f}s ({timings['get_tree']/timings['total']*100:.1f}%)")
            print(f"  ├─ Extract branches: {timings['extract_branches']:.3f}s ({timings['extract_branches']/timings['total']*100:.1f}%)")
            print(f"  ├─ Read batches: {timings['read_batches']:.3f}s ({timings['read_batches']/timings['total']*100:.1f}%) - {num_batches} batches")
            print(f"  ├─ Concatenate: {timings['concatenate']:.3f}s ({timings['concatenate']/timings['total']*100:.1f}%)")
            print(f"  └─ Zip objects: {timings['zip_objects']:.3f}s ({timings['zip_objects']/timings['total']*100:.1f}%)")
            print(f"  Memory used: {mem_used:.1f} MB")
            print(f"  Throughput: {result_dict['throughput_mb_per_sec']:.1f} MB/s")
            
            # Clean up
            del result
            gc.collect()
            
        except Exception as e:
            print(f"❌ Batch size {batch_size}: Error - {str(e)[:100]}")
            import traceback
            traceback.print_exc()
    
    return results

# Run tests on available files
all_results = []
import json
# test_files = json.load(open("/storage/agrp/netalev/data/batch_job_file_ids.json"))["2024r-pp"]
input_dir = "/storage/agrp/netalev/data/root_files/"  # Update this path if needed

for filename, size in test_files:
    file_path = os.path.join(input_dir, filename)

    if os.path.exists(file_path):
        results = test_batch_sizes(file_path, filename)
        all_results.extend(results)
    else:
        print(f"File not found: {file_path}")


False

Testing file: 2024r-pp_mc_b1c18ef224d66cb7.root
File size: 0.01 MB
Number of entries: 20,202,169

Batch size:   no_batch
  Total time: 25.73s
  ├─ Open file: 0.003s (0.0%)
  ├─ Get tree: 0.027s (0.1%)
  ├─ Extract branches: 0.000s (0.0%)
  ├─ Read batches: 25.372s (98.6%) - 1 batches
  ├─ Concatenate: 0.001s (0.0%)
  └─ Zip objects: 0.331s (1.3%)
  Memory used: 3895.5 MB
  Throughput: 0.0 MB/s
❌ Batch size 10000: Error - 'FileNotFoundError' object is not subscriptable
❌ Batch size 20000: Error - [Errno 2] No such file or directory: '/storage/agrp/netalev/data/root_files/2024r-pp_mc_b1c18ef224d6
❌ Batch size 40000: Error - [Errno 2] No such file or directory: '/storage/agrp/netalev/data/root_files/2024r-pp_mc_b1c18ef224d6


Traceback (most recent call last):
  File "/tmp/ipykernel_3952124/1889418538.py", line 36, in test_batch_sizes
    result, timings = parse_root_file_instrumented(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3952124/2388208990.py", line 60, in parse_root_file_instrumented
    batch_data = tree.arrays(all_branches, entry_start=entry_start, entry_stop=entry_stop)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv01/agrp/netalev/.local/lib/python3.11/site-packages/uproot/behaviors/TBranch.py", line 888, in arrays
    _ranges_or_baskets_to_arrays(
  File "/srv01/agrp/netalev/.local/lib/python3.11/site-packages/uproot/behaviors/TBranch.py", line 3198, in _ranges_or_baskets_to_arrays
    uproot.source.futures.delayed_raise(*obj)
  File "/srv01/agrp/netalev/.local/lib/python3.11/site-packages/uproot/source/futures.py", line 38, in delayed_raise
    raise exception_value.with_traceback(traceback)
  File "/srv01/

❌ Batch size 80000: Error - [Errno 2] No such file or directory: '/storage/agrp/netalev/data/root_files/2024r-pp_mc_b1c18ef224d6
❌ Batch size 160000: Error - [Errno 2] No such file or directory: '/storage/agrp/netalev/data/root_files/2024r-pp_mc_b1c18ef224d6
File not found: /storage/agrp/netalev/data/root_files/2024r-pp_0fced08cf06527b7.root
File not found: /storage/agrp/netalev/data/root_files/2024r-pp_4c4043706f0d183b.root


Traceback (most recent call last):
  File "/tmp/ipykernel_3952124/1889418538.py", line 36, in test_batch_sizes
    result, timings = parse_root_file_instrumented(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3952124/2388208990.py", line 19, in parse_root_file_instrumented
    with uproot.open(file_index) as root_file:
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv01/agrp/netalev/.local/lib/python3.11/site-packages/uproot/reading.py", line 142, in open
    file = ReadOnlyFile(
           ^^^^^^^^^^^^^
  File "/srv01/agrp/netalev/.local/lib/python3.11/site-packages/uproot/reading.py", line 573, in __init__
    self._begin_chunk = self._source.chunk(
                        ^^^^^^^^^^^^^^^^^^^
  File "/srv01/agrp/netalev/.local/lib/python3.11/site-packages/uproot/source/fsspec.py", line 95, in chunk
    data = self._fs.cat_file(self._file_path, start=start, end=stop)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv01/agrp/netal

In [ ]:
if all_results:
    df_results = pd.DataFrame(all_results)
    
    print("\n" + "="*80)
    print("PERFORMANCE SUMMARY")
    print("="*80)
    
    # Group by file and show comparison
    for filename in df_results['filename'].unique():
        file_df = df_results[df_results['filename'] == filename].copy()
        file_df = file_df.sort_values('total_time')
        
        print(f"\n📁 {filename} ({file_df.iloc[0]['file_size_mb']:.1f} MB, {file_df.iloc[0]['n_entries']:,} entries)")
        print("-" * 80)
        
        # Show best and worst
        best = file_df.iloc[0]
        worst = file_df.iloc[-1]
        
        print(f"🏆 Best: batch_size={best['batch_size']}, time={best['total_time']:.2f}s")
        print(f"   Read batches: {best['read_batches_time']:.2f}s ({best['read_batches_time']/best['total_time']*100:.1f}%)")
        print(f"   Concatenate: {best['concatenate_time']:.2f}s ({best['concatenate_time']/best['total_time']*100:.1f}%)")
        print(f"   Zip: {best['zip_objects_time']:.2f}s ({best['zip_objects_time']/best['total_time']*100:.1f}%)")
        
        print(f"\n❌ Worst: batch_size={worst['batch_size']}, time={worst['total_time']:.2f}s")
        print(f"   Read batches: {worst['read_batches_time']:.2f}s ({worst['read_batches_time']/worst['total_time']*100:.1f}%)")
        print(f"   Concatenate: {worst['concatenate_time']:.2f}s ({worst['concatenate_time']/worst['total_time']*100:.1f}%)")
        print(f"   Zip: {worst['zip_objects_time']:.2f}s ({worst['zip_objects_time']/worst['total_time']*100:.1f}%)")
        
        speedup = worst['total_time'] / best['total_time']
        print(f"\n⚡ Speedup: {speedup:.2f}x faster with best batch size")
    
    # Show detailed breakdown
    print("\n" + "="*80)
    print("DETAILED BREAKDOWN")
    print("="*80)
    print(df_results.to_string(index=False))
else:
    print("No results to analyze. Please run the test cells above.")


No results to analyze. Please run the test cells above.


## Visualize Performance


In [ ]:
if all_results:
    df_results = pd.DataFrame(all_results)
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Total time vs batch size
    ax1 = axes[0, 0]
    for filename in df_results['filename'].unique():
        file_df = df_results[df_results['filename'] == filename].copy()
        # Sort by batch_size, handling 'no_batch' string
        file_df['batch_size_num'] = file_df['batch_size'].apply(lambda x: float('inf') if x == 'no_batch' else x)
        file_df = file_df.sort_values('batch_size_num')
        
        batch_sizes_plot = [str(bs) if bs != 'no_batch' else 'no_batch' for bs in file_df['batch_size']]
        ax1.plot(batch_sizes_plot, file_df['total_time'], marker='o', label=filename[:30])
    
    ax1.set_xlabel('Batch Size')
    ax1.set_ylabel('Total Time (s)')
    ax1.set_title('Total Parse Time vs Batch Size')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    
    # 2. Time breakdown (stacked bar)
    ax2 = axes[0, 1]
    if len(df_results) > 0:
        # Take first file for detailed breakdown
        first_file = df_results['filename'].iloc[0]
        file_df = df_results[df_results['filename'] == first_file].copy()
        file_df = file_df.sort_values('total_time')
        
        batch_labels = [str(bs) if bs != 'no_batch' else 'no_batch' for bs in file_df['batch_size']]
        
        bottom = np.zeros(len(file_df))
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        for i, (col, color) in enumerate([
            ('open_file_time', colors[0]),
            ('get_tree_time', colors[1]),
            ('extract_branches_time', colors[2]),
            ('read_batches_time', colors[3]),
            ('concatenate_time', colors[4]),
            ('zip_objects_time', colors[5])
        ]):
            ax2.bar(batch_labels, file_df[col], bottom=bottom, label=col.replace('_time', ''), color=color, alpha=0.8)
            bottom += file_df[col]
        
        ax2.set_xlabel('Batch Size')
        ax2.set_ylabel('Time (s)')
        ax2.set_title(f'Time Breakdown by Step ({first_file[:30]})')
        ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax2.tick_params(axis='x', rotation=45)
    
    # 3. Read batches time vs number of batches
    ax3 = axes[1, 0]
    for filename in df_results['filename'].unique():
        file_df = df_results[df_results['filename'] == filename].copy()
        ax3.scatter(file_df['num_batches'], file_df['read_batches_time'], 
                   label=filename[:30], alpha=0.7, s=100)
    
    ax3.set_xlabel('Number of Batches')
    ax3.set_ylabel('Read Batches Time (s)')
    ax3.set_title('Read Time vs Number of Batches')
    ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax3.grid(True, alpha=0.3)
    
    # 4. Concatenate time vs number of batches
    ax4 = axes[1, 1]
    for filename in df_results['filename'].unique():
        file_df = df_results[df_results['filename'] == filename].copy()
        ax4.scatter(file_df['num_batches'], file_df['concatenate_time'], 
                   label=filename[:30], alpha=0.7, s=100)
    
    ax4.set_xlabel('Number of Batches')
    ax4.set_ylabel('Concatenate Time (s)')
    ax4.set_title('Concatenate Time vs Number of Batches')
    ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*80)
    print("KEY INSIGHTS")
    print("="*80)
    
    # Find bottlenecks
    for filename in df_results['filename'].unique():
        file_df = df_results[df_results['filename'] == filename].copy()
        avg_read = file_df['read_batches_time'].mean()
        avg_concat = file_df['concatenate_time'].mean()
        avg_zip = file_df['zip_objects_time'].mean()
        avg_total = file_df['total_time'].mean()
        
        print(f"\n📁 {filename}:")
        print(f"   Read batches: {avg_read:.2f}s ({avg_read/avg_total*100:.1f}% of total)")
        print(f"   Concatenate: {avg_concat:.2f}s ({avg_concat/avg_total*100:.1f}% of total)")
        print(f"   Zip objects: {avg_zip:.2f}s ({avg_zip/avg_total*100:.1f}% of total)")
        
        if avg_concat > avg_read * 0.5:
            print(f"   ⚠️  WARNING: Concatenate time is significant! This might be a bottleneck.")
        if avg_zip > avg_read * 0.3:
            print(f"   ⚠️  WARNING: Zip time is significant! This might be a bottleneck.")


## Test with Very Small Batch Sizes (Stress Test)


In [ ]:
# Test with very small batch sizes to see if batching overhead becomes significant
if test_files:
    filename, _ = test_files[0]  # Use first test file
    file_path = os.path.join(input_dir, filename)
    
    if os.path.exists(file_path):
        print(f"\n{'='*80}")
        print(f"STRESS TEST: Very Small Batch Sizes")
        print(f"{'='*80}")
        
        small_batch_sizes = [1_000, 5_000, 10_000, 20_000, 40_000]
        stress_results = test_batch_sizes(file_path, filename, batch_sizes=small_batch_sizes)
        
        if stress_results:
            df_stress = pd.DataFrame(stress_results)
            
            print("\n" + "="*80)
            print("STRESS TEST RESULTS")
            print("="*80)
            
            # Show how overhead scales with number of batches
            print("\nBatch Size | Num Batches | Total Time | Read Time | Concat Time | Overhead %")
            print("-" * 80)
            
            for _, row in df_stress.iterrows():
                overhead = (row['concatenate_time'] + row['zip_objects_time']) / row['total_time'] * 100
                print(f"{row['batch_size']:>10} | {row['num_batches']:>11} | {row['total_time']:>10.2f}s | "
                      f"{row['read_batches_time']:>9.2f}s | {row['concatenate_time']:>10.2f}s | {overhead:>9.1f}%")
            
            # Visualize
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
            
            # Time vs batch size
            ax1.plot(df_stress['batch_size'], df_stress['total_time'], marker='o', linewidth=2, markersize=8)
            ax1.set_xlabel('Batch Size')
            ax1.set_ylabel('Total Time (s)')
            ax1.set_title('Total Time vs Batch Size (Stress Test)')
            ax1.grid(True, alpha=0.3)
            
            # Overhead vs number of batches
            overhead_pct = (df_stress['concatenate_time'] + df_stress['zip_objects_time']) / df_stress['total_time'] * 100
            ax2.scatter(df_stress['num_batches'], overhead_pct, s=100, alpha=0.7)
            ax2.set_xlabel('Number of Batches')
            ax2.set_ylabel('Overhead % (Concat + Zip)')
            ax2.set_title('Batching Overhead vs Number of Batches')
            ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
    else:
        print(f"File not found: {file_path}")
else:
    print("No test files available.")



STRESS TEST: Very Small Batch Sizes
False

Testing file: 2024r-pp_mc_b1c18ef224d66cb7.root
File size: 0.01 MB
Number of entries: 20,202,169
❌ Batch size 1000: Error - AtlasOpenParser.extract_branches_by_obj_in_schema() got an unexpected keyword argument 'schema'


Traceback (most recent call last):
  File "/tmp/ipykernel_3931480/1153906297.py", line 36, in test_batch_sizes
    result, timings = parse_root_file_instrumented(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3931480/1496193027.py", line 34, in parse_root_file_instrumented
    obj_branches_and_quantities = parser.AtlasOpenParser.extract_branches_by_obj_in_schema(
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: AtlasOpenParser.extract_branches_by_obj_in_schema() got an unexpected keyword argument 'schema'


❌ Batch size 5000: Error - AtlasOpenParser.extract_branches_by_obj_in_schema() got an unexpected keyword argument 'schema'


Traceback (most recent call last):
  File "/tmp/ipykernel_3931480/1153906297.py", line 36, in test_batch_sizes
    result, timings = parse_root_file_instrumented(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3931480/1496193027.py", line 34, in parse_root_file_instrumented
    obj_branches_and_quantities = parser.AtlasOpenParser.extract_branches_by_obj_in_schema(
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: AtlasOpenParser.extract_branches_by_obj_in_schema() got an unexpected keyword argument 'schema'
Traceback (most recent call last):
  File "/tmp/ipykernel_3931480/1153906297.py", line 36, in test_batch_sizes
    result, timings = parse_root_file_instrumented(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3931480/1496193027.py", line 34, in parse_root_file_instrumented
    obj_branches_and_quantities = parser.AtlasOpenParser.extract_branches_by_obj_in_schema(
           

❌ Batch size 10000: Error - AtlasOpenParser.extract_branches_by_obj_in_schema() got an unexpected keyword argument 'schema'
❌ Batch size 20000: Error - AtlasOpenParser.extract_branches_by_obj_in_schema() got an unexpected keyword argument 'schema'
❌ Batch size 40000: Error - AtlasOpenParser.extract_branches_by_obj_in_schema() got an unexpected keyword argument 'schema'


Traceback (most recent call last):
  File "/tmp/ipykernel_3931480/1153906297.py", line 36, in test_batch_sizes
    result, timings = parse_root_file_instrumented(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3931480/1496193027.py", line 34, in parse_root_file_instrumented
    obj_branches_and_quantities = parser.AtlasOpenParser.extract_branches_by_obj_in_schema(
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: AtlasOpenParser.extract_branches_by_obj_in_schema() got an unexpected keyword argument 'schema'
